In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [24]:
data=pd.read_csv('최종 데이터(for추천).csv')
data.head()

,여행지,주소,행정구역,시군구,분류,트립어드바이저평점,카테고리
0,경복궁,서울특별시 종로구 사직로 161,서울시,종로구,역사적인 장소 • 역사박물관,4.5,역사적인 장소
1,삼청동길,"107 종로구 삼청동, 서울",서울시,종로구,즐길거리 & 랜드마크,4.5,랜드마크
2,인사동길,"종로구 인사동, 서울",서울시,종로구,인근 지역 • 즐길거리 & 랜드마크,4,랜드마크
3,조계사,"종로구 견지동 45번지, 서울",서울시,종로구,종교적인 장소 • 기념비 및 동상,4.5,종교적인 장소
4,청와대,"종로구 청와대로 1, 서울",서울시,종로구,공공기관 건물,4,랜드마크


In [25]:
data=data[['여행지','행정구역','시군구','트립어드바이저평점','카테고리']]
data.head()

,여행지,행정구역,시군구,트립어드바이저평점,카테고리
0,경복궁,서울시,종로구,4.5,역사적인 장소
1,삼청동길,서울시,종로구,4.5,랜드마크
2,인사동길,서울시,종로구,4,랜드마크
3,조계사,서울시,종로구,4.5,종교적인 장소
4,청와대,서울시,종로구,4,랜드마크


In [26]:
df=data.drop_duplicates(['여행지'])
df.shape

(652, 5)

In [27]:
df.head()

,여행지,행정구역,시군구,트립어드바이저평점,카테고리
0,경복궁,서울시,종로구,4.5,역사적인 장소
1,삼청동길,서울시,종로구,4.5,랜드마크
2,인사동길,서울시,종로구,4,랜드마크
3,조계사,서울시,종로구,4.5,종교적인 장소
4,청와대,서울시,종로구,4,랜드마크


In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 652 entries, 0 to 657
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   여행지        652 non-null    object
 1   행정구역       652 non-null    object
 2   시군구        652 non-null    object
 3   트립어드바이저평점  652 non-null    object
 4   카테고리       652 non-null    object
dtypes: object(5)
memory usage: 30.6+ KB


In [29]:
count_vector = CountVectorizer(ngram_range=(1,1))
c_vector1=count_vector.fit_transform(df['행정구역'])
c_vector2=count_vector.fit_transform(df['시군구'])
c_vector3=count_vector.fit_transform(df['카테고리'])

In [30]:
genre_c_sim1=cosine_similarity(c_vector1,c_vector1).argsort()[:,::-1]
genre_c_sim2=cosine_similarity(c_vector2,c_vector2).argsort()[:,::-1]
genre_c_sim3=cosine_similarity(c_vector3,c_vector3).argsort()[:,::-1]

In [31]:
genre_c_sim1.shape, genre_c_sim2.shape, genre_c_sim3.shape

((652, 652), (652, 652), (652, 652))

In [40]:
def get_recommend_item_list1(df,여행지명,top=652):
    target_item_index=df[df['여행지']==여행지명].index.values
    
    #코사인 유사도 중 비슷한 코사인 유사도를 가진 정보 추출
    sim_index1=genre_c_sim1[target_item_index,:].reshape(-1)
    sim_index2=genre_c_sim2[target_item_index,:].reshape(-1)
    sim_index3=genre_c_sim3[target_item_index,:].reshape(-1)
    
    #본인 제외
    sim_index1=sim_index1[sim_index1 != target_item_index]
    sim_index2=sim_index2[sim_index2 != target_item_index]
    sim_index3=sim_index3[sim_index3 != target_item_index]
    
    #data frame로 만들고 vote_count로 정렬한뒤 return
    #1 카테고리 2패턴재질 3모양 4계절 5스타일
    #우선순위 계절 카테고리 모양 스타일 패턴재질순
    result1=df.iloc[sim_index1].sort_values('트립어드바이저평점',ascending=True)
    result2=df.iloc[sim_index2].sort_values('트립어드바이저평점',ascending=True)
    result3=df.iloc[sim_index3].sort_values('트립어드바이저평점',ascending=True)

    
#     result4['항목']=5
#     result1['항목']=4
#     result3['항목']=3
#     result5['항목']=2
#     result2['항목']=1
    
    results=pd.concat([result1,result2,result3],ignore_index=True)
    results=results.sort_values('트립어드바이저평점',ascending=True)[:25]
#     results=results.sort_values(['종합점수','계절','카테고리','모양','스타일','패턴재질'],ascending=False)[:1000]
    results=results.drop_duplicates(['여행지'])
    #5개의 result에서 공통된 값들만 합침
    return results

In [41]:
get_recommend_item_list1(df,'경복궁')

,여행지,행정구역,시군구,트립어드바이저평점,카테고리
0,송천 떡마을,강원도,양양군,2.5,랜드마크
1305,화진포 해양박물관,강원도,고성군,2.5,박물관
1304,무이예술관,강원도,평창군,2.5,박물관
1302,그림이 있는 정원,충청남도,홍성군,2.5,랜드마크
655,호텔미란다 스파플러스,경기도,이천시,3,리조트
656,중앙탑과 충주박물관,충청북도,청주시,3,박물관
660,도피안사,강원도,철원군,3,역사적인 장소
661,마량리 동백숲,충청남도,서천시,3,자연 경관
1313,덕산 스파캐슬,충청남도,예산군,3,리조트
657,광양 장도박물관,전라남도,광양시,3,박물관
